In [1]:
print(1)

1


In [1]:
!pip install pandas geopy folium plotly nbformat

In [2]:
import pandas as pd
import requests
from io import StringIO, BytesIO
from zipfile import ZipFile
from geopy.distance import geodesic
import folium
import plotly.express as px
import nbformat

In [3]:
url = "http://download.geonames.org/export/dump/cities500.zip"
response = requests.get(url)
print(response)
print(response.content.__len__())


<Response [200]>
10575420


<Response [200]>


In [4]:

with ZipFile(BytesIO(response.content)) as zf:
    with zf.open('cities500.txt') as file:
        # Read the .txt file, but treat it as a CSV
        columns = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 
                   'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 
                   'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date']
        dtype_dict = {
            'geonameid': int,
            'name': str,
            'latitude': float,
            'longitude': float,
            'population': int
        }
        df = pd.read_csv(file, sep='\t', header=None, names=columns, dtype=dtype_dict, low_memory=False)
print(df)

        geonameid                 name            asciiname  \
0         3038999               Soldeu               Soldeu   
1         3039154            El Tarter            El Tarter   
2         3039163  Sant Julià de Lòria  Sant Julia de Loria   
3         3039604       Pas de la Casa       Pas de la Casa   
4         3039678               Ordino               Ordino   
...           ...                  ...                  ...   
199778     895269           Beitbridge           Beitbridge   
199779     895308             Beatrice             Beatrice   
199780     895417               Banket               Banket   
199781    1085510              Epworth              Epworth   
199782    1106542          Chitungwiza          Chitungwiza   

                                           alternatenames  latitude  \
0                                                     NaN  42.57688   
1                                    Ehl Tarter,Эл Тартер  42.57952   
2       San Julia,San Julià,Sa

In [5]:
#df[df['name'].str.contains('New York',na=False)]
nyc_entry=df[df['name'] == 'New York City'].iloc[0]
nyc=(nyc_entry['latitude'],nyc_entry['longitude'])
nyc
# Calculate distances and filter cities
df['distance_from_nyc'] = df.apply(lambda row: geodesic(nyc, (float(row['latitude']), float(row['longitude']))).miles, axis=1)


In [9]:
filtered_cities = df[(df['distance_from_nyc'] >= 7100) & (df['distance_from_nyc'] <= 7400) & (df['population'].astype(int) > 1000000)]
print(filtered_cities[['name', 'distance_from_nyc','population']])


                name  distance_from_nyc  population
17553     Lubumbashi        7340.665459     1373770
20898       Shangqiu        7222.313798     1536392
21060      Zhengzhou        7214.649726     4253913
21168         Anyang        7118.440115     1146839
21240        Kunshan        7379.329659     2092496
21489         Yixing        7393.645540     1285785
21583       Yangzhou        7327.665523     1584237
21608   Zhangjiagang        7351.353875     1432044
21681       Yancheng        7254.808553     1615717
21745        Xuchang        7263.111077     1265536
21842        Xinyang        7392.608059     1230042
21851       Xinxiang        7184.086301     1047088
21915         Xining        7101.821158     1677177
22212       Xianyang        7263.423079     1034081
22285          Xi’an        7268.396866     9600000
22352           Wuxi        7374.254725     4396835
22494         Weinan        7250.050454     1199290
22721       Tianshui        7249.601650     1212791
22862       

                name  distance_from_nyc  population
17553     Lubumbashi        7340.665459     1373770
20898       Shangqiu        7222.313798     1536392
21060      Zhengzhou        7214.649726     4253913
21168         Anyang        7118.440115     1146839
21240        Kunshan        7379.329659     2092496
21489         Yixing        7393.645540     1285785
21583       Yangzhou        7327.665523     1584237
21608   Zhangjiagang        7351.353875     1432044
21681       Yancheng        7254.808553     1615717
21745        Xuchang        7263.111077     1265536
21842        Xinyang        7392.608059     1230042
21851       Xinxiang        7184.086301     1047088
21915         Xining        7101.821158     1677177
22212       Xianyang        7263.423079     1034081
22285          Xi’an        7268.396866     9600000
22352           Wuxi        7374.254725     4396835
22494         Weinan        7250.050454     1199290
22721       Tianshui        7249.601650     1212791
22862       

In [8]:
# Create a map centered around NYC
m = folium.Map(location=[40.7128, -74.0060], zoom_start=2)

# Add markers for each city
for index, row in filtered_cities.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

# Save the map to an HTML file (or display it in a Jupyter notebook)
m.save('filtered_cities_map.html')

In [16]:
# Use Plotly to create a scatter plot on a 3D globe
fig = px.scatter_geo(filtered_cities,
                     lat='latitude',
                     lon='longitude',
                     projection="orthographic",
                     hover_name="name",
                     size="population",
                     size_max=50,
                     title="Cities 7200 to 7400 miles from NYC with Population > 500k")

# Show the figure
fig.show()

In [17]:
filtered_cities["population"].describe()

count    6.000000e+01
mean     2.805775e+06
std      3.501437e+06
min      1.006557e+06
25%      1.280723e+06
50%      1.564802e+06
75%      2.548449e+06
max      2.231547e+07
Name: population, dtype: float64

In [10]:
filtered_cities.dtypes

geonameid              int64
name                  object
asciiname             object
alternatenames        object
latitude             float64
longitude            float64
feature class         object
feature code          object
country code          object
cc2                   object
admin1 code           object
admin2 code           object
admin3 code           object
admin4 code           object
population             int64
elevation            float64
dem                    int64
timezone              object
modification date     object
distance_from_nyc    float64
dtype: object